In [5]:
%load_ext autoreload
%autoreload 2

from hossam.util import *
from hossam.plot import *
from hossam.tensor import *
from pandas import merge, read_excel, pivot_table, melt, to_datetime, to_timedelta, DateOffset, Timestamp, Period, Series
import numpy as np
import re
import pandas as pd
from datetime import datetime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
origin = my_read_excel("분할.xlsx")
df = origin.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10030 entries, 0 to 10029
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   회원카드번호  10030 non-null  int64         
 1   연월      10030 non-null  datetime64[ns]
 2   등록명     6501 non-null   object        
 3   매출액     10030 non-null  int64         
 4   미수액     10030 non-null  int64         
 5   개월수     10030 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 470.3+ KB
None

상위 5개 행
+----+----------------+---------------------+----------+----------+----------+----------+
|    |   회원카드번호 | 연월                | 등록명   |   매출액 |   미수액 |   개월수 |
|----+----------------+---------------------+----------+----------+----------+----------|
|  0 |       12131325 | 2020-09-01 00:00:00 | 헬스     |   899000 |        0 |       12 |
|  1 |       13110008 | 2021-05-01 00:00:00 | 골프     |  1700000 |        0 |       12 |
|  2 |       13110008 | 2021-

### 휴면가능 기간 처리

In [7]:
df['휴면가능'] = np.where(df['개월수'] >= 36, 6, np.where(df['개월수'] >= 24, 4, np.where(df['개월수'] >= 12, 2, 0)))
df

,회원카드번호,연월,등록명,매출액,미수액,개월수,휴면가능
0,12131325,2020-09-01,헬스,899000,0,12,2
1,13110008,2021-05-01,골프,1700000,0,12,2
2,13110008,2021-06-01,NaN,200000,0,0,0
3,13110008,2022-06-01,골프,1700000,0,12,2
4,13110008,2023-08-01,NaN,220000,0,0,0
...,...,...,...,...,...,...,...
10025,23120077,2023-12-01,헬스,990000,0,10,0
10026,24010001,2023-12-01,헬스,990000,0,10,0
10027,24010002,2023-12-01,NaN,990000,0,0,0
10028,24010003,2022-01-01,헬스,1035000,0,10,0


In [8]:
df['개월수'].unique()

array([ 12,   0, -12,  24,   5,  10,  14,   6,   1,   3, -10, -14,   2,
         4,   9,  -3,  11,  -6, -11,  -9,   7,  -1,   8,  21,  -5, -24,
        36,  13,  15,  30,  18,  -8,  -7], dtype=int64)

In [9]:
for i in range(len(df)):
    # Convert Period to Timestamp, add DateOffset, and then convert back to Period
    df.loc[i, '종료시점'] = (df.loc[i, '연월'] + DateOffset(months=df.loc[i, '개월수']))
    df.loc[i, '휴면반영'] = (df.loc[i, '연월'] + DateOffset(months=df.loc[i, '개월수'] + df.loc[i, '휴면가능']))
df

,회원카드번호,연월,등록명,매출액,미수액,개월수,휴면가능,종료시점,휴면반영
0,12131325,2020-09-01,헬스,899000,0,12,2,2021-09-01,2021-11-01
1,13110008,2021-05-01,골프,1700000,0,12,2,2022-05-01,2022-07-01
2,13110008,2021-06-01,NaN,200000,0,0,0,2021-06-01,2021-06-01
3,13110008,2022-06-01,골프,1700000,0,12,2,2023-06-01,2023-08-01
4,13110008,2023-08-01,NaN,220000,0,0,0,2023-08-01,2023-08-01
...,...,...,...,...,...,...,...,...,...
10025,23120077,2023-12-01,헬스,990000,0,10,0,2024-10-01,2024-10-01
10026,24010001,2023-12-01,헬스,990000,0,10,0,2024-10-01,2024-10-01
10027,24010002,2023-12-01,NaN,990000,0,0,0,2023-12-01,2023-12-01
10028,24010003,2022-01-01,헬스,1035000,0,10,0,2022-11-01,2022-11-01


In [10]:
df1 = df[['회원카드번호', '연월']].groupby(by= '회원카드번호', as_index =False).min()
df1.rename(columns={'연월':'최초등록월'}, inplace=True)
df2 = merge(df, df1, on =['회원카드번호'])
df2

,회원카드번호,연월,등록명,매출액,미수액,개월수,휴면가능,종료시점,휴면반영,최초등록월
0,12131325,2020-09-01,헬스,899000,0,12,2,2021-09-01,2021-11-01,2020-09-01
1,13110008,2021-05-01,골프,1700000,0,12,2,2022-05-01,2022-07-01,2021-05-01
2,13110008,2021-06-01,NaN,200000,0,0,0,2021-06-01,2021-06-01,2021-05-01
3,13110008,2022-06-01,골프,1700000,0,12,2,2023-06-01,2023-08-01,2021-05-01
4,13110008,2023-08-01,NaN,220000,0,0,0,2023-08-01,2023-08-01,2021-05-01
...,...,...,...,...,...,...,...,...,...,...
10025,23120077,2023-12-01,헬스,990000,0,10,0,2024-10-01,2024-10-01,2023-12-01
10026,24010001,2023-12-01,헬스,990000,0,10,0,2024-10-01,2024-10-01,2023-12-01
10027,24010002,2023-12-01,NaN,990000,0,0,0,2023-12-01,2023-12-01,2023-12-01
10028,24010003,2022-01-01,헬스,1035000,0,10,0,2022-11-01,2022-11-01,2022-01-01


In [11]:
df2['등록명'].unique()
등록명 = ['헬스', '골프','골프골프', '헬스헬스', '헬스골프', '헬스헬스헬스', '골프헬스',
       '헬스골프골프', '골프골프골프', '골프골프골프골프', '골프헬스헬스', '골프헬스골프골프']
df2 = df2[df2['등록명'].isin(등록명)]
df2.reset_index(drop=True, inplace=True)

In [12]:
def diff_month(d1, d2):

    return (d1.year - d2.year) * 12 + d1.month - d2.month

In [13]:
# 회원카드
회원카드 = df2['회원카드번호'].unique()
행병합 = []

for i in 회원카드:
    a = df2.query('회원카드번호 == @i').sort_values(by="연월")  # 연월에 대해 데이터프레임을 정렬
    a.reset_index(drop=True, inplace=True)
    
    for j in range(len(a)) :
        a.loc[j, '거래순서'] = j  # 거래순서 열 추가 # pandas ranking 써 볼
        a.loc[j, "마지막거래"] = len(a)-1
        개월수 = 0
        
        for k in range (j+1) :            
            개월수 += a.loc[k, '개월수']
        
        if j == 0:
            a.loc[j, '잔여개월수'] = a.loc[j, "개월수"]
            a.loc[j, '종료시점'] = a.loc[j,"연월"] + DateOffset(months = 개월수)
            a.loc[j, "결제시점"] = 0
            
        if (j > 0) and a.loc[j, '개월수'] :
            a.loc[j, '종료시점'] =  a.loc[j-1,"연월"] + DateOffset(months = 개월수)
            a.loc[j, "잔여개월수"] = diff_month(a.loc[j, '종료시점'], a.loc[j,'연월'])
            a.loc[j, "결제시점"] = diff_month(a.loc[j-1, '종료시점'], a.loc[j,'연월'])
            
            if to_datetime(a.loc[j,"연월"]) <= to_datetime('2020-12-01'):  # 코로나 이전    
                if a.loc[j, "결제시점"] < 0 and a.loc[j-1, "휴면가능"] != 0:
                    휴면보정값 = min(a.loc[j-1, "휴면가능"], abs(a.loc[j, "결제시점"]))
                
                    if 휴면보정값 < 0:
                        휴면보정값 = a.loc[j-1, "휴면가능"]  # 휴면가능을 기존 값으로 고정
                
                    a.loc[j-1, '휴면반영'] = (a.loc[j-1, '연월'] + DateOffset(months=a.loc[j-1, '개월수'] + 휴면보정값))
                # 종료시점을 휴면반영으로 설정
                    a.loc[j-1, '종료시점'] = a.loc[j-1, '휴면반영']
                    a.loc[j-1, '휴면반영'] = None
                
                a.loc[j-1, '잔여개월수'] = diff_month(a.loc[j-1, '종료시점'] + DateOffset(months=1), a.loc[j-1, '연월'])
                a.loc[j, "결제시점"] = diff_month(a.loc[j-1, '종료시점'], a.loc[j, '연월'])

            elif to_datetime(a.loc[j,"연월"]) <= to_datetime('2021-01-01'):  # 코로나 기간
                if a.loc[j, "결제시점"] < 0 and a.loc[j-1, "휴면가능"] != 0:
                    휴면보정값 = min(a.loc[j-1, "휴면가능"], abs(a.loc[j-1, "결제시점"]))
                
                    if 휴면보정값 < 0:
                        휴면보정값 = a.loc[j-1, "휴면가능"]  # 휴면가능을 기존 값으로 고정
                
                    a.loc[j-1, '휴면반영'] = (a.loc[j-1, '연월'] + DateOffset(months=a.loc[j-1, '개월수'] + 휴면보정값))
                
                # 종료시점을 휴면반영으로 설정
                    a.loc[j-1, '종료시점'] = a.loc[j-1, '휴면반영']
                    a.loc[j-1, '휴면반영'] = None
                
                # 휴면가능한 경우 휴면가능 수를 추가하여 잔여 개월수를 다시 계산
                a.loc[j-1, '잔여개월수'] = diff_month(a.loc[j-1, '종료시점'] + DateOffset(months=1), a.loc[j-1,'연월'])
                a.loc[j, "결제시점"] = diff_month(a.loc[j-1, '종료시점'], a.loc[j,'연월'])
                
                
            elif to_datetime(a.loc[j,"연월"]) <= to_datetime('2023-01-01'):  # 코로나와 리모델링 사이
                if a.loc[j, "결제시점"] < 0 and a.loc[j-1, "휴면가능"] != 0:
                    휴면보정값 = min(a.loc[j-1, "휴면가능"], abs(a.loc[j, "결제시점"]))
                
                    if 휴면보정값 < 0:
                        휴면보정값 = a.loc[j-1, "휴면가능"]  # 휴면가능을 기존 값으로 고정

                    a.loc[j-1, '휴면반영'] = (a.loc[j-1, '연월'] + DateOffset(months=a.loc[j-1, '개월수'] + 휴면보정값))
                
                # 종료시점을 휴면반영으로 설정
                    a.loc[j-1, '종료시점'] = a.loc[j-1, '휴면반영']
                    a.loc[j-1, '휴면반영'] = None
                
                # 휴면가능한 경우 휴면가능 수를 추가하여 잔여 개월수를 다시 계산
                a.loc[j-1, '잔여개월수'] = diff_month(a.loc[j-1, '종료시점'], a.loc[j-1,'연월'])
                a.loc[j, "결제시점"] = diff_month(a.loc[j-1, '종료시점'], a.loc[j,'연월'])
            elif to_datetime(a.loc[j,"연월"]) <= to_datetime('2023-02-01'):  # 리모델링 기간
                if a.loc[j, "결제시점"] < 0 and a.loc[j-1, "휴면가능"] != 0:
                    휴면보정값 = min(a.loc[j-1, "휴면가능"], abs(a.loc[j-1, "결제시점"]))
                    if 휴면보정값 < 0:
                        휴면보정값 = a.loc[j-1, "휴면가능"]  # 휴면가능을 기존 값으로 고정
                        a.loc[j-1, '휴면반영'] = (a.loc[j-1, '연월'] + DateOffset(months=a.loc[-1j, '개월수'] + 휴면보정값))
                
                # 종료시점을 휴면반영으로 설정
                    a.loc[j-1, '종료시점'] = a.loc[j-1, '휴면반영']
                    a.loc[j-1, '휴면반영'] = None
                
                # 휴면가능한 경우 휴면가능 수를 추가하여 잔여 개월수를 다시 계산
                a.loc[j-1, '잔여개월수'] = diff_month(a.loc[j-1, '종료시점'] + DateOffset(months=2), a.loc[j-1,'연월'])
                a.loc[j, "결제시점"] = diff_month(a.loc[j-1, '종료시점'], a.loc[j,'연월'])
            else:  # 리모델링 이후
                if a.loc[j, "결제시점"] < 0 and a.loc[j-1, "휴면가능"] != 0:
                    휴면보정값 = min(a.loc[j-1, "휴면가능"], abs(a.loc[j, "결제시점"]))
                
                    if 휴면보정값 < 0:
                        휴면보정값 = a.loc[j-1, "휴면가능"]  # 휴면가능을 기존 값으로 고정
                
                    a.loc[j-1, '휴면반영'] = (a.loc[j-1, '연월'] + DateOffset(months=a.loc[j-1, '개월수'] + 휴면보정값))
                
                # 종료시점을 휴면반영으로 설정
                    a.loc[j-1, '종료시점'] = a.loc[j-1, '휴면반영']
                    a.loc[j-1, '휴면반영'] = None
                
                a.loc[j-1, '잔여개월수'] = diff_month(a.loc[j-1, '종료시점'], a.loc[j-1,'연월'])
                a.loc[j, "결제시점"] = diff_month(a.loc[j-1, '종료시점'], a.loc[j,'연월'])
                         
        if a.loc[j, '결제시점'] < 0 :
            a.loc[j, "이탈여부"] = 1
        else:
            a.loc[j, "이탈여부"] = 0
        
        if (a.loc[j, "결제시점"] == a.loc[j, "마지막거래"]) and (a.loc[j,'종료시점'] < to_datetime("2024-01", format="%Y-%m")):
            a.loc[j, "이탈여부"] = 1
            
        elif (a.loc[j,"거래순서"] == a.loc[j, "마지막거래"]) and (a.loc[j,'종료시점'] >= to_datetime("2024-01", format="%Y-%m")):
            a.loc[j, "이탈여부"] = 2
           
            # a.loc[j, '잔여개월수'] = a.loc[j,"종료시점"] - a.loc[j, "연월"]
    행병합.append(a)  # 행병합 리스트에 데이터프레임 추가

# 모든 데이터프레임을 행 방향으로 병합
df3 = concat(행병합, ignore_index=True)
df3

,회원카드번호,연월,등록명,매출액,미수액,개월수,휴면가능,종료시점,휴면반영,최초등록월,거래순서,마지막거래,잔여개월수,결제시점,이탈여부
0,12131325,2020-09-01,헬스,899000,0,12,2,2021-09-01,2021-11-01,2020-09-01,0.0,0.0,12.0,0.0,1.0
1,13110008,2021-05-01,골프,1700000,0,12,2,2022-06-01,NaT,2021-05-01,0.0,2.0,13.0,0.0,0.0
2,13110008,2022-06-01,골프,1700000,0,12,2,2023-08-01,NaT,2021-05-01,1.0,2.0,14.0,0.0,0.0
3,13110008,2023-11-01,골프,2120000,0,12,2,2025-06-01,2025-01-01,2021-05-01,2.0,2.0,19.0,-3.0,2.0
4,13110009,2020-10-01,헬스,1140000,0,12,2,2021-12-01,NaT,2020-10-01,0.0,2.0,14.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6495,23120076,2023-12-01,헬스,990000,0,10,0,2024-10-01,2024-10-01,2023-12-01,0.0,0.0,10.0,0.0,2.0
6496,23120077,2023-12-01,헬스,990000,0,10,0,2024-10-01,2024-10-01,2023-12-01,0.0,0.0,10.0,0.0,2.0
6497,24010001,2023-12-01,헬스,990000,0,10,0,2024-10-01,2024-10-01,2023-12-01,0.0,0.0,10.0,0.0,2.0
6498,24010003,2022-01-01,헬스,1035000,0,10,0,2022-11-01,2022-11-01,2022-01-01,0.0,0.0,10.0,0.0,1.0


In [14]:
df3 = df3.drop(['개월수','휴면가능','휴면반영','최초등록월','거래순서','마지막거래'],axis=1)
df3

,회원카드번호,연월,등록명,매출액,미수액,종료시점,잔여개월수,결제시점,이탈여부
0,12131325,2020-09-01,헬스,899000,0,2021-09-01,12.0,0.0,1.0
1,13110008,2021-05-01,골프,1700000,0,2022-06-01,13.0,0.0,0.0
2,13110008,2022-06-01,골프,1700000,0,2023-08-01,14.0,0.0,0.0
3,13110008,2023-11-01,골프,2120000,0,2025-06-01,19.0,-3.0,2.0
4,13110009,2020-10-01,헬스,1140000,0,2021-12-01,14.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
6495,23120076,2023-12-01,헬스,990000,0,2024-10-01,10.0,0.0,2.0
6496,23120077,2023-12-01,헬스,990000,0,2024-10-01,10.0,0.0,2.0
6497,24010001,2023-12-01,헬스,990000,0,2024-10-01,10.0,0.0,2.0
6498,24010003,2022-01-01,헬스,1035000,0,2022-11-01,10.0,0.0,1.0


In [15]:
df3['연월'] = df3['연월'].dt.strftime('%Y-%m')  # 연월 열의 일(day) 부분을 없애기
df3['종료시점'] = df3['종료시점'].dt.strftime('%Y-%m')  # 종료시점 열의 일(day) 부분을 없애기
df3

,회원카드번호,연월,등록명,매출액,미수액,종료시점,잔여개월수,결제시점,이탈여부
0,12131325,2020-09,헬스,899000,0,2021-09,12.0,0.0,1.0
1,13110008,2021-05,골프,1700000,0,2022-06,13.0,0.0,0.0
2,13110008,2022-06,골프,1700000,0,2023-08,14.0,0.0,0.0
3,13110008,2023-11,골프,2120000,0,2025-06,19.0,-3.0,2.0
4,13110009,2020-10,헬스,1140000,0,2021-12,14.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
6495,23120076,2023-12,헬스,990000,0,2024-10,10.0,0.0,2.0
6496,23120077,2023-12,헬스,990000,0,2024-10,10.0,0.0,2.0
6497,24010001,2023-12,헬스,990000,0,2024-10,10.0,0.0,2.0
6498,24010003,2022-01,헬스,1035000,0,2022-11,10.0,0.0,1.0


### 문자발송 내역, 출석추세선 통합


In [16]:
df4 = read_excel("문자발송.xlsx")
df4

,연월,광고
0,2020-11,1
1,2020-06,1
2,2020-03,1
3,2020-02,1
4,2020-01,1
5,2021-11,1
6,2021-06,1
7,2022-11,1
8,2022-06,1
9,2022-05,1


In [17]:
df5 = read_excel("통합출석추세선.xlsx")
df5

,회원카드번호,연월,추세값
0,20010027,2020-01,-0.690909
1,20010004,2020-01,-0.490909
2,20010024,2020-01,-0.581818
3,19080042,2020-01,1.136364
4,20010048,2020-01,0.254545
...,...,...,...
2093,15060067,2023-12,-9.000000
2094,18080035,2023-12,4.000000
2095,13120098,2023-12,10.000000
2096,17030079,2023-12,-16.000000


In [18]:
origin = merge(df3, df4, how="left", on="연월")
origin['광고'] = origin['광고'].fillna(0)
origin

,회원카드번호,연월,등록명,매출액,미수액,종료시점,잔여개월수,결제시점,이탈여부,광고
0,12131325,2020-09,헬스,899000,0,2021-09,12.0,0.0,1.0,0.0
1,13110008,2021-05,골프,1700000,0,2022-06,13.0,0.0,0.0,0.0
2,13110008,2022-06,골프,1700000,0,2023-08,14.0,0.0,0.0,1.0
3,13110008,2023-11,골프,2120000,0,2025-06,19.0,-3.0,2.0,0.0
4,13110009,2020-10,헬스,1140000,0,2021-12,14.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
6495,23120076,2023-12,헬스,990000,0,2024-10,10.0,0.0,2.0,0.0
6496,23120077,2023-12,헬스,990000,0,2024-10,10.0,0.0,2.0,0.0
6497,24010001,2023-12,헬스,990000,0,2024-10,10.0,0.0,2.0,0.0
6498,24010003,2022-01,헬스,1035000,0,2022-11,10.0,0.0,1.0,1.0


In [19]:
origin1 = origin.copy()
origin1 = merge(origin1, df5, how="left", on=("회원카드번호","연월"))
origin1['추세값'] = origin1['추세값'].fillna(0)
origin1

,회원카드번호,연월,등록명,매출액,미수액,종료시점,잔여개월수,결제시점,이탈여부,광고,추세값
0,12131325,2020-09,헬스,899000,0,2021-09,12.0,0.0,1.0,0.0,-0.131868
1,13110008,2021-05,골프,1700000,0,2022-06,13.0,0.0,0.0,0.0,-0.263736
2,13110008,2022-06,골프,1700000,0,2023-08,14.0,0.0,0.0,1.0,-0.285714
3,13110008,2023-11,골프,2120000,0,2025-06,19.0,-3.0,2.0,0.0,3.500000
4,13110009,2020-10,헬스,1140000,0,2021-12,14.0,0.0,0.0,0.0,0.840659
...,...,...,...,...,...,...,...,...,...,...,...
6495,23120076,2023-12,헬스,990000,0,2024-10,10.0,0.0,2.0,0.0,0.000000
6496,23120077,2023-12,헬스,990000,0,2024-10,10.0,0.0,2.0,0.0,0.000000
6497,24010001,2023-12,헬스,990000,0,2024-10,10.0,0.0,2.0,0.0,0.000000
6498,24010003,2022-01,헬스,1035000,0,2022-11,10.0,0.0,1.0,1.0,0.000000


In [20]:
origin1.to_excel("변수통합.xlsx",index=False)